## Import used packages

In [ ]:
import pandas as pd
import sys
from datetime import date, datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import spatial
from math import sin, cos, sqrt, atan2, radians
import folium
from tqdm import tqdm
sys.path.insert(0,"..")

## Import data

In [ ]:
bremen_trips = pd.read_csv("../data/processed/trips_weather.csv", index_col=0)
bremen_trips = bremen_trips.sort_values(by=['start_time'], ascending=True)
bremen_trips["start_time"] = pd.to_datetime(bremen_trips["start_time"])

In [ ]:
bremen_trips.head(2)

## Functions that calculates distance moved towards the university of Bremen and main station

In [ ]:
# Function that calculates if start or end location is closer to University of Bremen. 
# Returns the difference of the distances of the start and end location in kilometers.
# If value positive, end locations is closer to university. That means moved towards university.
def distanceToUni(sLng,sLat,eLng,eLat):
    # approximate radius of earth in km
    R = 6373.0
    
    sLng = radians(sLng)
    sLat = radians(sLat)
    eLng = radians(eLng)
    eLat = radians(eLat)
    uLat = radians(53.1069302) # University of Bremen Latitude
    uLng = radians(8.8499603)  # University of Bremen Longitude

    sdlon = uLng - sLng
    sdlat = uLat - sLat

    edlon = uLng - eLng
    edlat = uLat - eLat

    sA = sin(sdlat / 2)**2 + cos(sLat) * cos(uLat) * sin(sdlon / 2)**2
    eA = sin(edlat / 2)**2 + cos(eLat) * cos(uLat) * sin(edlon / 2)**2
    
    sC = 2 * atan2(sqrt(sA), sqrt(1 - sA))
    eC = 2 * atan2(sqrt(eA), sqrt(1 - eA))

    sDist = R * sC
    eDist = R * eC

    distance = sDist - eDist
    
    return distance

def distanceToMainStation(sLng,sLat,eLng,eLat):
    # approximate radius of earth in km
    R = 6373.0
    
    sLng = radians(sLng)
    sLat = radians(sLat)
    eLng = radians(eLng)
    eLat = radians(eLat)
    msLat = radians(53.083122) # # Main station Latitude
    msLng = radians(8.813717) # # Main station Latitude

    sdlon = msLng - sLng
    sdlat = msLat - sLat

    edlon = msLng - eLng
    edlat = msLat - eLat

    sA = sin(sdlat / 2)**2 + cos(sLat) * cos(msLat) * sin(sdlon / 2)**2
    eA = sin(edlat / 2)**2 + cos(eLat) * cos(msLat) * sin(edlon / 2)**2
    
    sC = 2 * atan2(sqrt(sA), sqrt(1 - sA))
    eC = 2 * atan2(sqrt(eA), sqrt(1 - eA))

    sDist = R * sC
    eDist = R * eC

    distance = sDist - eDist
    
    return distance

## Example of trip moving away from the university

- Red marker = University of Bremen
- Blue marker = Start location
- Green marker = End location

In [ ]:
m = folium.Map(location=[53.1069302,8.8499603], zoom_start=13)

folium.Marker([53.1069302,8.8499603], popup='<i>Uni</i>',icon=folium.Icon(color='red')).add_to(m)
folium.Marker([53.083122,8.813717], popup='<i>Main Station</i>',icon=folium.Icon(color='red')).add_to(m)

row = 35
folium.Marker([bremen_trips.iloc[row,5],bremen_trips.iloc[row,4]], popup='<i>Start</i>').add_to(m)
folium.Marker([bremen_trips.iloc[row,7],bremen_trips.iloc[row,6]], popup='<i>End</i>',icon=folium.Icon(color='green')).add_to(m)

print('Moved km in direction to uni: ', distanceToUni(bremen_trips.iloc[row,4],bremen_trips.iloc[row,5],bremen_trips.iloc[row,6],bremen_trips.iloc[row,7]))
print('Moved km in direction to main station: ', distanceToMainStation(bremen_trips.iloc[row,4],bremen_trips.iloc[row,5],bremen_trips.iloc[row,6],bremen_trips.iloc[row,7]))

m

## Add to_uni, to_main_station and direction attribute to trips data

In [ ]:
to_uni = []
to_main_station = []
to_uni_bool = []
to_main_station_bool = []
month = []
week_day = []
is_weekend = []
hour = []


for index, row in tqdm(bremen_trips.iterrows()):
    
    dist_to_uni = distanceToUni(row['start_lng'],row['start_lat'],row['end_lng'],row['end_lat'])
    dist_to_main_station = distanceToMainStation(row['start_lng'],row['start_lat'],row['end_lng'],row['end_lat'])
    
    day = row['start_time'].weekday()
    
    # Save datetime information
    month.append(row['start_time'].month)
    week_day.append(day)
    hour.append(row['start_time'].hour)
    
    if (day == 5) | (day == 6):
        is_weekend.append(1)
    else:
        is_weekend.append(0)
    
    # Save distances to corresponding list
    to_uni.append(dist_to_uni)
    to_main_station.append(dist_to_main_station)
    
    if dist_to_uni < 0:
        to_uni_bool.append(0)
    else:
        to_uni_bool.append(1)
        
    if dist_to_main_station < 0:
        to_main_station_bool.append(0)
    else:
        to_main_station_bool.append(1)
        

bremen_trips['to_uni'] = to_uni
bremen_trips['to_main_station'] = to_main_station
bremen_trips['to_uni_bool'] = to_uni_bool
bremen_trips['to_main_station_bool'] = to_main_station_bool
bremen_trips['month'] = month
bremen_trips['week_day'] = week_day
bremen_trips['is_weekend'] = is_weekend
bremen_trips['hour'] = hour

In [ ]:
bremen_trips.sample(3)

## Check on correlations to_uni

In [ ]:
bremen_trips.drop(columns={'end_lng','end_lat','end_place','to_uni','to_uni_bool','to_main_station_bool','to_main_station','duration_sec','identification'}).corrwith(bremen_trips['to_uni'])

### Pearson correlation only for attributes with correlation higher than 0.1

In [ ]:
#Using Pearson Correlation

plt.figure(figsize=(10,8))
cor = bremen_trips[['to_uni','start_lng','start_lat','start_place']].corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

## Check on correlations to_main_station

In [ ]:
bremen_trips.drop(columns={'end_lng','end_lat','end_place','to_uni_bool','to_main_station_bool','to_uni','to_main_station','duration_sec','identification'}).corrwith(bremen_trips['to_main_station'])

### Pearson correlation only for attributes with correlation higher than 0.1

In [ ]:
#Using Pearson Correlation

plt.figure(figsize=(10,8))
cor = bremen_trips[['to_main_station','start_lng','start_place','humidity_2m']].corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

## Check on correlations to_main_station

In [ ]:
bremen_trips.drop(columns={'end_lng','end_lat','end_place','to_uni','to_uni_bool','to_main_station_bool','to_main_station','duration_sec','identification'}).corrwith(bremen_trips['to_uni_bool'])

### Pearson correlation only for attributes with correlation higher than 0.1

In [ ]:
#Using Pearson Correlation

plt.figure(figsize=(10,8))
cor = bremen_trips[['to_uni_bool','start_lng','start_place','humidity_2m','month']].corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

# Predicting to_uni_bool - Classification

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Initialize independent and target variable
X = bremen_trips[['start_lng','start_lat','start_place','humidity_2m','month']]
y = bremen_trips['to_uni_bool']

In [ ]:
# Splitting data into train and validation set
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.60, shuffle=True, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.60, shuffle=True, random_state=42)

In [ ]:
rf = RandomForestClassifier(criterion='entropy', class_weight='balanced_subsample')
rf.fit(X_train, y_train)

### Evaluation

In [ ]:
print('Training accuracy: ' + str(rf.score(X_train, y_train)))
print('Valiation accuracy: ' + str(rf.score(X_val, y_val)))

In [ ]:
rf.score(X_val, y_val)

In [ ]:
rf.predict_proba(X_val)

In [ ]:
# Function that plots a confusion matrix given independent and target variable
def confusionMatrix(y, X):
    cm = confusion_matrix(y_val, rf.predict(X_val))

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(cm)
    ax.grid(False)
    ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0s', 'Predicted 1s'))
    ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0s', 'Actual 1s'))
    ax.set_ylim(1.5, -0.5)
    for i in range(2):
        for j in range(2):
            ax.text(j, i, cm[i, j], ha='center', va='center', color='red')
    plt.show()

In [ ]:
confusionMatrix(y_val, rf.predict(X_val))

In [ ]:
print(classification_report(y_val, rf.predict(X_val)))

### Performing grid search to optimize hyperparameters

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
parameters = {'max_depth':[2, 3, 4, 5, 7],
              'n_estimators':[1, 10, 25, 50, 100, 256, 512],
              'random_state':[42]}
    
def perform_grid_search(X_data, y_data):
    rf = RandomForestClassifier(criterion='entropy', class_weight='balanced_subsample')
    
    clf = GridSearchCV(rf, parameters, cv=4, scoring='accuracy', n_jobs=3)
    
    clf.fit(X_data, y_data)
    
    print(clf.cv_results_['mean_test_score'])
    
    return clf.best_params_['n_estimators'], clf.best_params_['max_depth']

In [ ]:
# extract parameters
n_estimator, depth = perform_grid_search(X_val, y_val)
c_random_state = 42
print(n_estimator, depth, c_random_state)

In [ ]:
rf = RandomForestClassifier(criterion='entropy', class_weight='balanced_subsample', n_estimators=n_estimator, max_depth=depth)
rf.fit(X_train, y_train)

### Evaluation

In [ ]:
print('Training accuracy: ' + str(rf.score(X_train, y_train)))
print('Valiation accuracy: ' + str(rf.score(X_val, y_val)))

In [ ]:
confusionMatrix(y_val, rf.predict(X_val))

In [ ]:
print(classification_report(y_val, rf.predict(X_val)))

## Apply to test set

### Add validation set to train set to obtain more training data

In [ ]:
X_train = X_train.append(X_val)
y_train = y_train.append(y_val)

In [ ]:
rf = RandomForestClassifier(criterion='entropy', class_weight='balanced_subsample', n_estimators=n_estimator, max_depth=depth)
rf.fit(X_train, y_train)

### Training + validation set performance

In [ ]:
print('Training accuracy: ' + str(rf.score(X_train, y_train)))

### Test set performance

### The moment of truth

In [ ]:
print('Test accuracy: ' + str(rf.score(X_test, y_test)))

In [ ]:
print(classification_report(y_test, rf.predict(X_test)))

# Try Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Initialize independent and target variable
X = bremen_trips[['start_lng','start_lat','start_plz','month']]
y = bremen_trips['to_uni_bool']

In [ ]:
# Splitting data into train and validation set
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.60, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.60, shuffle=True)

In [ ]:
model = LogisticRegression(solver='liblinear',multi_class='ovr', random_state=42, class_weight='balanced').fit(X_train, y_train)

In [ ]:
print('Training accuracy: ' + str(model.score(X_train, y_train)))
print('Valiation accuracy: ' + str(model.score(X_val, y_val)))

In [ ]:
model.predict_proba(X_val)

In [ ]:
confusionMatrix(y_val, model.predict(X_val))

In [ ]:
print(classification_report(y_val, model.predict(X_val)))

### Logistic regression performs worst than random forest. Do not use it!

# Predicting to_main_station_bool - Classification

In [ ]:
# Initialize independent and target variable
X = bremen_trips[['start_lng','start_lat','start_plz','month']]
y = bremen_trips['to_main_station_bool']

In [ ]:
# Splitting data into train and validation set
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.60, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.60, shuffle=True)

In [ ]:
rf = RandomForestClassifier(criterion='entropy', class_weight='balanced_subsample')
rf.fit(X_train, y_train)

### Evaluation

In [ ]:
print('Training accuracy: ' + str(rf.score(X_train, y_train)))
print('Valiation accuracy: ' + str(rf.score(X_val, y_val)))

In [ ]:
rf.predict_proba(X_val)

In [ ]:
confusionMatrix(y_val, rf.predict(X_val))

In [ ]:
print(classification_report(y_val, rf.predict(X_val)))

### Performing grid search to optimize hyperparameters

In [ ]:
parameters = {'max_depth':[2, 3, 4, 5, 7],
              'n_estimators':[1, 10, 25, 50, 100, 256, 512],
              'random_state':[42]}
    
def perform_grid_search(X_data, y_data):
    rf = RandomForestClassifier(criterion='entropy', class_weight='balanced_subsample')
    
    clf = GridSearchCV(rf, parameters, cv=4, scoring='accuracy', n_jobs=3)
    
    clf.fit(X_data, y_data)
    
    print(clf.cv_results_['mean_test_score'])
    
    return clf.best_params_['n_estimators'], clf.best_params_['max_depth']

In [ ]:
# extract parameters
n_estimator, depth = perform_grid_search(X_val, y_val)
c_random_state = 42
print(n_estimator, depth, c_random_state)

In [ ]:
rf = RandomForestClassifier(criterion='entropy', class_weight='balanced_subsample', n_estimators=n_estimator, max_depth=depth)
rf.fit(X_train, y_train)

### Evaluation

In [ ]:
print('Training accuracy: ' + str(rf.score(X_train, y_train)))
print('Valiation accuracy: ' + str(rf.score(X_val, y_val)))

In [ ]:
confusionMatrix(y_val, rf.predict(X_val))

In [ ]:
print(classification_report(y_val, rf.predict(X_val)))

## Apply to test set

### Add validation set to train set to obtain more training data

In [ ]:
X_train = X_train.append(X_val)
y_train = y_train.append(y_val)

In [ ]:
rf = RandomForestClassifier(criterion='entropy', class_weight='balanced_subsample', n_estimators=n_estimator, max_depth=depth)
rf.fit(X_train, y_train)

### Training + validation set performance

In [ ]:
print('Training accuracy: ' + str(rf.score(X_train, y_train)))

### Test set performance

### The moment of truth

In [ ]:
print('Test accuracy: ' + str(rf.score(X_test, y_test)))

In [ ]:
print(classification_report(y_test, rf.predict(X_test)))

# Predicting to_uni - Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

In [ ]:
# Initialize independent and target variable
X = bremen_trips[['start_lng','start_lat','start_place']]
y = bremen_trips['to_uni']

### Polynomial regression

In [ ]:
# Splitting data into train and validation set
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.60, shuffle=True, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.60, shuffle=True, random_state=42)

In [ ]:
def chooseDegree(X, y):
    
    # Splitting data into train and validation set
    #X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.60, shuffle=True)
    #X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.60, shuffle=True)

    r2_train = []
    r2_val = []
    rmses_val = []
    degrees = np.arange(1, 8)
    min_rmse, min_deg = 1e10, 0

    for deg in degrees:

        # Train features
        poly_features = PolynomialFeatures(degree=deg, include_bias=False)
        X_poly_train = poly_features.fit_transform(X_train)

        # Linear regression
        poly_reg = LinearRegression()
        poly_reg.fit(X_poly_train, y_train)
        X_val_poly = poly_features.fit_transform(X_val)
        
        # Evaluate
        r2_train.append(poly_reg.score(X_poly_train, y_train))
        r2_val.append(poly_reg.score(X_val_poly,y_val))

        # Compare with val data
        poly_predict = poly_reg.predict(X_val_poly)
        poly_mse = mean_squared_error(y_val, poly_predict)
        poly_rmse = np.sqrt(poly_mse)
        rmses_val.append(poly_rmse)

        # Cross-validation of degree
        if min_rmse > poly_rmse:
            min_rmse = poly_rmse
            min_deg = deg

    # Plot and present results
    print('Suggested degree {} with RMSE of validation set {}'.format(min_deg, min_rmse))
    
    # Create model with optimal degree
    poly_features = PolynomialFeatures(degree=min_deg, include_bias=False)
    X_poly_train = poly_features.fit_transform(X_train)

    # Linear regression
    poly_reg = LinearRegression()
    poly_reg.fit(X_poly_train, y_train)
    X_val_poly = poly_features.fit_transform(X_val)

    # Evaluate
    print('R2 train score with suggested degree: ', poly_reg.score(X_poly_train, y_train))
    print('R2 validation score with suggested degree: ', poly_reg.score(X_val_poly,y_val))
    
    
    # Evaluation plots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))
    
    #ax_rmses_val = fig.add_subplot(111)
    axes[0].plot(degrees, rmses_val, label='rmses_val')
    axes[0].set_yscale('log')
    axes[0].set_xlabel('Degree')
    axes[0].set_ylabel('RMSE')
    
    #ax_r2_train = fig.add_subplot(111)
    axes[1].plot(degrees, r2_train, label='R2_train')
    axes[1].set_yscale('linear')
    axes[1].set_xlabel('Degree')
    axes[1].set_ylabel('R2_train')
    axes[1].set_label('R2_train')
    #axes[1].set_ylim(0,1)
    
    #ax_r2_val = fig.add_subplot(111)
    axes[1].plot(degrees, r2_val, label='R2_val')
    axes[1].set_xlabel('Degree')
    axes[1].set_ylabel('R2_val')
    
    axes[0].legend()
    axes[1].legend()
    fig.tight_layout()

In [ ]:
chooseDegree(X,y)

## Apply to test set

### Add validation set to train set to obtain more training data

In [ ]:
X_train = X_train.append(X_val)
y_train = y_train.append(y_val)

poly = PolynomialFeatures(3)
X_train_poly = poly.fit_transform(X_train)

lin = LinearRegression()

### Training + validation set performance

In [ ]:
lin.fit(X_train_poly, y_train)
lin.score(X_train_poly, y_train)

### Test set performance

In [ ]:
X_test_poly = poly.fit_transform(X_test)
lin.score(X_test_poly,y_test)

# Predicting to_main_station - Regression

In [ ]:
# Initialize independent and target variable
X = bremen_trips.dropna()[['start_lng','start_place','start_place','humidity_2m']]
y = bremen_trips.dropna()['to_main_station']

In [ ]:
# Splitting data into train and validation set
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.60, shuffle=True, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.60, shuffle=True, random_state=42)

In [ ]:
chooseDegree(X,y)

## Apply to test set

### Add validation set to train set to obtain more training data

In [ ]:
X_train = X_train.append(X_val)
y_train = y_train.append(y_val)

poly = PolynomialFeatures(9)
X_train_poly = poly.fit_transform(X_train)

lin = LinearRegression()

### Training + validation set performance

In [ ]:
lin.fit(X_train_poly, y_train)
lin.score(X_train_poly, y_train)

### Test set performance

In [ ]:
X_test_poly = poly.fit_transform(X_test)
lin.score(X_test_poly,y_test)